In [12]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast
import glob

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
gbg = pb.retrosheet.season_game_logs(2008)

In [6]:
seasons = []
for x in range(2008,2020,1):
    seasons.append(x)

In [7]:
seasons.pop()

2019

In [8]:
seasons

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [9]:
def season_pull(year):
    name = 'overall_'+str(year)
    '''Pull from retrosheet for season indicated by year 
    EX: season_pull(2008) returns a DataFrame containing 
    the entire game by game for the 2008 MLB Season'''
    df=pb.retrosheet.season_game_logs(year)
    df.to_csv('C:/Users/dakot/Documents/GitHub/baseball_predict/Data/'+name+'.csv' )
    return df

In [10]:
for x in seasons:
    season_pull(x)

In [25]:
df = pd.read_csv('C:/Users/dakot/Documents/GitHub/baseball_predict/Data/overall_2008.csv')
cols = []
for x in df.columns:
    cols.append(x)

In [ ]:
# cols.pop(0)

In [39]:
path = 'baseball_predict/Data/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename,usecols=cols,index_col=None)
    li.append(df)

frame = pd.concat(li, axis=0)

In [45]:
frame.shape

(26726, 161)

In [48]:
frame.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,visiting_score,home_score,num_outs,day_night,completion_info,forfeit_info,protest_info,park_id,attendance,time_of_game_minutes,visiting_line_score,home_line_score,visiting_abs,visiting_hits,visiting_doubles,visiting_triples,visiting_homeruns,visiting_rbi,visiting_sac_hits,visiting_sac_flies,visiting_hbp,visiting_bb,visiting_iw,visiting_k,visiting_sb,visiting_cs,visiting_gdp,visiting_ci,visiting_lob,visiting_pitchers_used,visiting_individual_er,visiting_er,visiting__wp,visiting_balks,visiting_po,visiting_assists,visiting_errors,visiting_pb,visiting_dp,visiting_tp,home_abs,home_hits,home_doubles,home_triples,home_homeruns,home_rbi,home_sac_hits,home_sac_flies,home_hbp,home_bb,home_iw,home_k,home_sb,home_cs,home_gdp,home_ci,home_lob,home_pitchers_used,home_individual_er,home_er,home_wp,home_balks,home_po,home_assists,home_errors,home_pb,home_dp,home_tp,ump_home_id,ump_home_name,ump_first_id,ump_first_name,ump_second_id,ump_second_name,ump_third_id,ump_third_name,ump_lf_id,ump_lf_name,ump_rf_id,ump_rf_name,visiting_manager_id,visiting_manager_name,home_manager_id,home_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,save_pitcher_id,save_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,visiting_starting_pitcher_id,visiting_starting_pitcher_name,home_starting_pitcher_id,home_starting_pitcher_name,visiting_1_id,visiting_1_name,visiting_1_pos,visiting_2_id,visiting_2_name,visiting_2_pos,visiting_2_id.1,visiting_3_name,visiting_3_pos,visiting_4_id,visiting_4_name,visiting_4_pos,visiting_5_id,visiting_5_name,visiting_5_pos,visiting_6_id,visiting_6_name,visiting_6_pos,visiting_7_id,visiting_7_name,visiting_7_pos,visiting_8_id,visiting_8_name,visiting_8_pos,visiting_9_id,visiting_9_name,visiting_9_pos,home_1_id,home_1_name,home_1_pos,home_2_id,home_2_name,home_2_pos,home_3_id,home_3_name,home_3_pos,home_4_id,home_4_name,home_4_pos,home_5_id,home_5_name,home_5_pos,home_6_id,home_6_name,home_6_pos,home_7_id,home_7_name,home_7_pos,home_8_id,home_8_name,home_8_pos,home_9_id,home_9_name,home_9_pos,misc,acquisition_info
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,6,5,60,N,NaN,NaN,NaN,TOK01,44628.0,219,0000030012,2000020001,38,11,3,0,1,6,1,0,0,3,2,7,0,0,2,0,6,6,5,5,1,0,30,15,0,0,0,0,36,7,1,0,2,5,0,0,1,7,0,10,1,0,0,0,9,5,6,6,0,0,30,12,0,0,2,0,reedr901,Rick Reed,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,okajh001,Hideki Okajima,streh001,Huston Street,papej001,Jonathan Papelbon,ramim002,Manny Ramirez,matsd001,Daisuke Matsuzaka,blanj001,Joe Blanton,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,ellsj001,Jacoby Ellsbury,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,custj001,Jack Cust,10,browe001,Emil Brown,7,crosb002,Bobby Crosby,6,hannj001,Jack Hannahan,5,suzuk001,Kurt Suzuki,2,sweer001,Ryan Sweeney,8,NaN,Y
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,1,5,51,N,NaN,NaN,NaN,TOK01,44735.0,171,000001000,01300001x,31,5,1,0,1,1,0,0,0,3,0,13,1,0,2,0,6,5,5,5,0,0,24,8,0,0,0,0,33,9,3,0,1,5,0,0,0,4,0,7,0,0,0,0,8,4,1,1,1,0,27,5,1,0,2,0,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,reedr901,Rick Reed,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,hardr001,Rich Harden,lestj001,Jon Lester,NaN,(none),denoc001,Chris Denorfia,lestj001,Jon Lester,hardr001,Rich Harden,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,crisc001,Coco Crisp,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,sweem002,Mike Sweeney,10,browe001,Emil Brown,7,crosb002,Bobby Crosby

In [76]:
for x in frame:
    frame['win'] = np.where(frame['home_score']>frame['visiting_score'],frame['home_team'],frame['visiting_team'])

In [79]:
frame.win.describe()

count     26726
unique       31
top         NYA
freq       1010
Name: win, dtype: object